In [6]:
import pandas as pd

# Dados Gerais
dg_bairros = pd.read_csv("database_csv/bairro.csv")
dg_sub_bacias = pd.read_csv("database_csv/sub_bacias.csv")
dg_grid = pd.read_csv("database_csv/h3_grid_res8.csv")

# # Dados de COR, 1746, e pontos supervisionados
cor_ocorrencias = pd.read_csv("database_csv/ocorrencias.csv")
cor_ocorrencias_pop_filtrado = pd.read_csv("database_csv/ocorrencias_pop_filtrado.csv")
cor_pop = pd.read_csv("database_csv/procedimento_operacional_padrao.csv")
datario_chamado_1746_filtrado = pd.read_csv("database_csv/chamado_2.csv")
ra_psalagamento = pd.read_csv("database_csv/ponto_supervisionado_alagamento.csv")

# # Dados Pluviométricos
cor_est_alerta_rio = pd.read_csv("database_csv/estacoes_alertario.csv")
cor_est_inea = pd.read_csv("database_csv/estacoes_inea.csv")
cor_est_cemaden = pd.read_csv("database_csv/estacoes_cemaden.csv")
cor_est_websirene = pd.read_csv("database_csv/estacoes_websirene.csv")

cor_tx_precip_websirene = pd.read_csv("database_csv/taxa_precipitacao_websirene.csv")
cor_tx_precip_alerta_rio_5_min = pd.read_csv("database_csv/taxa_precipitacao_alertario_5min.csv")
cor_tx_precip_inea = pd.read_csv("database_csv/taxa_precipitacao_inea.csv")
cor_tx_precip_cemaden = pd.read_csv("database_csv/taxa_precipitacao_cemaden.csv")

# # Dados Fluviométricos e dados de reservtórios
ra_lam_inea = pd.read_csv("database_csv/lamina_agua_inea.csv")
ra_lam_via = pd.read_csv("database_csv/nivel_lamina_agua_via.csv")
ra_nivel_reservatorio = pd.read_csv("database_csv/nivel_reservatorio.csv")

# # Radares_metereológicos
# cor_tx_precip_guaratiba = pd.read_csv("database_csv/taxa_precipitacao_guaratiba.csv")

/tmp/ipykernel_1888/1183382940.py:12: DtypeWarning: Columns (21,22,29) have mixed types. Specify dtype option on import or set low_memory=False.
  datario_chamado_1746_filtrado = pd.read_csv("database_csv/chamado_2.csv")


In [ ]:
# Como criar um ranking de chuvas?

# Filtrando o dataset para os dias nos quais houveram chuvas
# cor_tx_precip_alerta_rio = cor_tx_precip_alerta_rio[(cor_tx_precip_alerta_rio["acumulado_chuva_24h"]>0)]

# Ordenando os dados por data
cor_tx_precip_alerta_rio = cor_tx_precip_alerta_rio.sort_values(by='data_medicao', ascending=False)

# # Transformando os dados em mm/h

colunas_acumulado_chuva = ['acumulado_chuva_5min', 'acumulado_chuva_10min', 'acumulado_chuva_15min',
                           'acumulado_chuva_30min', 'acumulado_chuva_1h', 'acumulado_chuva_2h',
                           'acumulado_chuva_3h', 'acumulado_chuva_4h', 'acumulado_chuva_6h',
                           'acumulado_chuva_12h', 'acumulado_chuva_24h']

# Converter os acumulados de chuva para mm/h
for coluna in colunas_acumulado_chuva:
    # Extrair o período de tempo da coluna
    tempo_str = coluna.split('_')[2]
    if tempo_str[-3:] == "min":
        tempo = int(tempo_str[:-3]) / 60
    else:
        tempo = int(tempo_str[:-1])
    
    # Converter para mm/h
    cor_tx_precip_alerta_rio[coluna + '_mmh'] = cor_tx_precip_alerta_rio[coluna] / tempo


cor_tx_precip_alerta_rio['data_medicao'] = pd.to_datetime(cor_tx_precip_alerta_rio['data_medicao'])
dias_unicos = pd.unique(cor_tx_precip_alerta_rio['data_medicao'].dt.date)

In [10]:
cor_tx_precip_alerta_rio.shape

(1983284, 9)

In [ ]:
# transformando para data
# Adicionando a Data
cor_tx_precip_websirene["data_medicao"] = cor_tx_precip_websirene['primary_key'].apply(lambda x: x.split('_')[1])
# cor_tx_precip_websirene["hora_medicao"] = cor_tx_precip_websirene['primary_key'].apply(lambda x: x.split()[1] if len(x.split()) >= 2 else None)
cor_tx_precip_websirene["data_medicao"] =  pd.to_datetime(cor_tx_precip_websirene["data_medicao"])
# cor_tx_precip_websirene["hora_medicao"] =  pd.to_datetime(cor_tx_precip_websirene["hora_medicao"])

# Ordenando por valor
cor_tx_precip_websirene = cor_tx_precip_websirene.sort_values(by='data_medicao', ascending=False)

# Lista das colunas que precisam ser corrigidas
colunas_a_corrigir = ['acumulado_chuva_15_min', 'acumulado_chuva_1_h', 'acumulado_chuva_4_h', 'acumulado_chuva_24_h', 'acumulado_chuva_96_h']

# Substituir NaN por 0 nas colunas especificadas
cor_tx_precip_websirene[colunas_a_corrigir] = cor_tx_precip_websirene[colunas_a_corrigir].fillna(0)

# cor_tx_precip_websirene = cor_tx_precip_websirene[cor_tx_precip_websirene["id_estacao"] == 5]

# Evento chuvoso
def criar_evento_chuvoso(df):
    eventos = {}
    evento_num = 0
    is_evento = False
    for index, row in df.iterrows():
        id_estacao = row['id_estacao']
        acumulado_4h = row['acumulado_chuva_4_h']
        if acumulado_4h == 0:
            is_evento = False
        else:
            if not is_evento:
                evento_num += 1
            evento_atual = f"{id_estacao}_evento_{evento_num}"
            is_evento = True
            eventos[index] = evento_atual
    return eventos

# Criar a coluna 'evento_chuvoso' usando a função criar_evento_chuvoso
eventos = criar_evento_chuvoso(cor_tx_precip_websirene)
cor_tx_precip_websirene['evento_chuvoso'] = pd.Series(eventos)

# Preencher os valores faltantes com 0
# cor_tx_precip_websirene['evento_chuvoso'].fillna(0, inplace=True)

# Exibir o DataFrame resultante
cor_tx_precip_websirene[cor_tx_precip_websirene["evento_chuvoso"] == "5_evento_1"]



,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_medicao,evento_chuvoso


In [ ]:
cor_tx_precip_websirene[cor_tx_precip_websirene["id_estacao"] == 5]

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_medicao,evento_chuvoso
1701273,5_2024-03-12 21:50:00,5,0.0,0.0,0.0,0.2,14.0,21:50:00,2024-03-12,2024-03-12 21:50:00,NaN
1899165,5_2024-03-12 21:40:00,5,0.0,0.0,0.0,0.2,14.0,21:40:00,2024-03-12,2024-03-12 21:40:00,NaN
809914,5_2024-03-12 21:25:00,5,0.0,0.0,0.0,0.2,14.0,21:25:00,2024-03-12,2024-03-12 21:25:00,NaN
908735,5_2024-03-12 21:15:00,5,0.0,0.0,0.0,0.2,14.0,21:15:00,2024-03-12,2024-03-12 21:15:00,NaN
1502931,5_2024-03-12 20:55:00,5,0.0,0.0,0.0,0.6,14.0,20:55:00,2024-03-12,2024-03-12 20:55:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1495234,5_2012-01-15 15:45:00,5,5.6,5.6,5.6,5.6,5.6,15:45:00,2012-01-15,2012-01-15 15:45:00,5_evento_168358
1148897,5_2012-01-14 20:00:00,5,2.6,2.6,2.6,2.6,2.6,20:00:00,2012-01-14,2012-01-14 20:00:00,5_evento_168379
156651,5_2012-01-14 19:45:00,5,1.0,1.0,1.0,1.0,1.0,19:45:00,2012-01-14,2012-01-14 19:45:00,5_evento_168380
702985,5_2012-01-13 16:00:00,5,0.2,0.2,0.2,0.2,0.2,16:00:00,2012-01-13,2012-01-13 16:00:00,5_evento_168381


In [7]:
datario_tx_precip_alerta_rio

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao
0,41_2023-11-14 21:45:00,41,0.8,1.0,1.0,3.0,3.0,21:45:00,2023-11-14
1,67_2023-11-14 23:45:00,67,NaN,14.2,15.8,15.8,16.0,23:45:00,2023-11-14
2,39_2023-11-14 23:00:00,39,2.4,11.2,11.6,11.6,11.6,23:00:00,2023-11-14
3,5_2023-11-14 23:15:00,5,1.2,10.8,10.8,11.0,15.2,23:15:00,2023-11-14
4,28_2023-11-14 23:15:00,28,1.8,6.6,6.8,6.8,7.2,23:15:00,2023-11-14
...,...,...,...,...,...,...,...,...,...
1983279,70_2012-05-31 12:15:00,70,0.0,0.2,4.6,4.6,4.6,12:15:00,2012-05-31
1983280,8_2012-05-31 12:00:00,8,0.0,2.0,2.2,2.2,2.2,12:00:00,2012-05-31
1983281,8_2012-05-31 11:00:00,8,0.0,0.2,0.2,0.2,0.2,11:00:00,2012-05-31
1983282,50_2012-05-31 23:00:00,50,0.0,0.0,0.0,0.4,0.4,23:00:00,2012-05-31
